In [15]:
import numpy as np
import pandas
import pandas as pd

In [16]:
import rbot


In [17]:
ftx = rbot.FtxMarket("BTC-PERP")

In [18]:
trades = ftx.select_trades(rbot.NOW() - rbot.DAYS(30), 0)

In [19]:
trades

,price,size,side,liquid
timestamp,,,,
2022-10-05 10:42:59.439493888+00:00,20132.0,2.2455,Sell,False
2022-10-05 10:42:59.439493888+00:00,20132.0,0.4967,Sell,False
2022-10-05 10:42:59.439493888+00:00,20132.0,0.4500,Sell,False
2022-10-05 10:42:59.439493888+00:00,20132.0,0.2000,Sell,False
2022-10-05 10:42:59.439493888+00:00,20132.0,0.4000,Sell,False
...,...,...,...,...
2022-11-04 01:39:09.116549888+00:00,20276.0,0.0001,Buy,False
2022-11-04 01:39:10.506121984+00:00,20276.0,0.0075,Buy,False
2022-11-04 01:39:12.834531072+00:00,20275.0,0.0001,Sell,False


In [ ]:
ohlcv = ftx.ohlcvv(rbot.NOW() - rbot.DAYS(15), 0, 60)

In [34]:
ohlcv

,order_side,open,high,low,close,vol,count
timestamp,,,,,,,
2022-10-25 10:45:50+00:00,Buy,19283.0,19283.0,19283.0,19283.0,0.0059,2.0
2022-10-25 10:45:50+00:00,Sell,19282.0,19282.0,19282.0,19282.0,0.0003,2.0
2022-10-25 10:46:00+00:00,Buy,19283.0,19283.0,19283.0,19283.0,0.0066,4.0
2022-10-25 10:46:00+00:00,Sell,19282.0,19282.0,19282.0,19282.0,0.0001,1.0
2022-10-25 10:46:10+00:00,Buy,19283.0,19283.0,19283.0,19283.0,0.0009,3.0
...,...,...,...,...,...,...,...
2022-11-04 01:39:00+00:00,Buy,20276.0,20276.0,20276.0,20276.0,0.0014,5.0
2022-11-04 01:39:00+00:00,Sell,20275.0,20275.0,20275.0,20275.0,0.0009,5.0
2022-11-04 01:39:10+00:00,Buy,20276.0,20276.0,20276.0,20276.0,0.0075,1.0


In [15]:
df['diff'] = df.index.to_series().diff()

In [8]:
ohlcv = ftx.ftx.ohlcvv(0, 0, 120)


In [10]:
ohlcv = pd.DataFrame(ohlcv)

In [12]:
ohlcv

,0,1,2,3,4,5,6,7
0,1.662285e+15,1.0,19791.0,19796.0,19791.0,19796.0,21.8226,88.0
1,1.662285e+15,0.0,19790.0,19792.0,19790.0,19792.0,0.1857,7.0
2,1.662285e+15,1.0,19804.0,19814.0,19796.0,19805.0,105.5337,274.0
3,1.662285e+15,0.0,19795.0,19813.0,19795.0,19804.0,20.3326,136.0
4,1.662285e+15,1.0,19805.0,19805.0,19794.0,19794.0,12.8460,82.0
...,...,...,...,...,...,...,...,...
87343,1.667526e+15,0.0,20265.0,20275.0,20264.0,20275.0,39.0043,117.0
87344,1.667526e+15,1.0,20276.0,20281.0,20276.0,20281.0,17.5999,94.0
87345,1.667526e+15,0.0,20275.0,20280.0,20275.0,20280.0,18.8817,84.0
87346,1.667526e+15,1.0,20281.0,20281.0,20276.0,20276.0,0.6292,18.0


In [18]:
array = ftx.select_trades(0, 0)

In [19]:
array.shape

(26068123, 5)

In [20]:
df = trades_to_df(array)

KeyError: 'side'

In [34]:
df

,price,size,side,liquid
timestamp,,,,
2022-09-13 13:30:21.208737024+00:00,21337.0,0.0028,Sell,True
2022-09-13 13:36:34.898341120+00:00,21374.0,0.0401,Sell,True
2022-09-13 13:36:47.153015040+00:00,21360.0,0.0432,Sell,True
2022-09-13 13:37:45.171756032+00:00,21329.0,0.2658,Sell,True
2022-09-13 13:38:14.774477056+00:00,21314.0,0.0002,Sell,True
...,...,...,...,...
2022-11-02 10:04:01.318811904+00:00,20400.0,0.2713,Sell,True
2022-11-02 10:06:33.648379904+00:00,20373.0,0.0267,Sell,True
2022-11-02 10:06:42.642800896+00:00,20361.0,0.1708,Sell,True


In [47]:
df2 = pd.read_csv('ftx.csv', names=("timestamp", 'price', 'order_side', 'liquid', 'id'), i)

In [48]:
df2

,timestamp,price,order_side,liquid,id
1667314934485667,20391,0.0001,Sell,false,5283994328
1667314934380211,20391,0.0001,Sell,false,5283994317
1667314934341958,20392,0.2453,Buy,false,5283994311
1667314934286755,20392,0.0260,Buy,false,5283994301
1667314934271012,20391,0.0001,Sell,false,5283994291
...,...,...,...,...,...
1667227934951002,20350,0.0100,Buy,false,5274339831
1667227934951002,20350,0.0010,Buy,false,5274339830
1667227934439297,20349,0.0010,Sell,false,5274339780
1667227934285876,20349,0.0015,Buy,false,5274339748


In [39]:
df2['diff'] = df2['timestamp'].diff()

In [40]:
df2['diff']

1667314934485667    NaN
1667314934380211    0.0
1667314934341958    1.0
1667314934286755    0.0
1667314934271012   -1.0
                   ... 
1667227934951002    0.0
1667227934951002    0.0
1667227934439297   -1.0
1667227934285876    0.0
1667227934285876    0.0
Name: diff, Length: 293968, dtype: float64

In [45]:
df2[df2['diff'] < -1]

,timestamp,price,order_side,liquid,id,diff
1667314810143806,20405,0.0001,Sell,false,5283979059,-2.0
1667314729898654,20388,0.0011,Buy,false,5283972144,-2.0
1667314723949493,20389,0.1000,Buy,false,5283971641,-2.0
1667314723930986,20388,0.2390,Buy,false,5283971630,-3.0
1667314721484732,20383,0.0200,Buy,false,5283971168,-2.0
...,...,...,...,...,...,...
1667227982525311,20348,1.0762,Buy,false,5274349198,-2.0
1667227981683112,20345,0.0010,Buy,false,5274349097,-2.0
1667227961829144,20346,0.0010,Buy,false,5274345177,-2.0
1667227960847934,20344,1.6144,Buy,false,5274344978,-2.0
